In [3]:
from __future__ import division

import json
import pickle
import pandas as pd
import numpy as np
import math
from scipy.ndimage.interpolation import shift

In [16]:


with open('../data/general/matches_2017_2018_v1.json') as matches_json:
    matches_json = pd.read_json(matches_json)
    
matches = [60565]


no_team = 0
home_team = 0
away_team = 0
match_count = 1



data_height = 6000 # number of seconds in per match
data_width = 128 # number of features for fector


for id in matches:
    with open('../data/match_' + str(id) + '/match_data_' + str(id) + '.json', 'r') as file:
        team = json.load(file)
      
        
    # defining 3 distinct id in game
    referee_id = 0
    home_team_id = int(team[1]['teamId'])
    away_team_id = int(team[2]['teamId'])
    
    # home player list
    home_team_player = []
    away_team_player = []
    
    # player dicts
    home_dict = {} 
    away_dict = {} 
    
    # players position ID 
    player_positionId = {'homeTeam' : {}, 'awayTeam' : {}}
    
    
    with open('../data/match_' + str(id) + '/roster_data_' + str(id) + '.json', 'r') as file:
        match_squad = json.load(file)
        
    for roaster in match_squad:
        if roaster['teamId'] == home_team_id:
            home_team_player.append(str(roaster['jerseyNumber']))
            player_positionId['homeTeam'][roaster['jerseyNumber']] = roaster['positionId'] 
            
        elif roaster['teamId'] == away_team_id:
            away_team_player.append(str(roaster['jerseyNumber']))
            player_positionId['awayTeam'][roaster['jerseyNumber']] = roaster['positionId']
            
    
    home_team_player.sort(key=int)
    home_val = np.arange(0, len(home_team_player))
    home_dict = dict(zip(home_team_player, home_val))

    away_team_player.sort(key=int)
    away_val = np.arange(0, len(away_team_player))
    away_dict = dict(zip(away_team_player, away_val))
    
    
    # home and away team player pos and activity count: 0-xpos, 1-ypos
    home_pos_count = np.zeros(shape=(len(home_team_player), 2), dtype=np.float)
    home_activity_count = np.zeros(shape=(len(home_team_player), 1), dtype=np.int)
    away_pos_count = np.zeros(shape=(len(away_team_player), 2), dtype=np.float)
    away_activity_count = np.zeros(shape=(len(away_team_player), 1), dtype=np.int)
 

    # home and away team player count data for 5 min: 0-xpos, 1-ypos / 0-1min, 1-2min... 4-5min 
    home_pos_count_5min = np.zeros(shape=(len(home_team_player), 2, 5), dtype=np.float)
    home_activity_count_5min = np.zeros(shape=(len(home_team_player), 5), dtype=np.int)
    
    away_pos_count_5min = np.zeros(shape=(len(away_team_player), 2, 5), dtype=np.float)
    away_activity_count_5min = np.zeros(shape=(len(away_team_player), 5), dtype=np.int)
    
    
    # home and away team Mean pos data: 0-xpos, 1-ypos
    home_pos_mean = np.zeros(shape=(len(home_team_player), 2), dtype=np.float)
    
    away_pos_mean = np.zeros(shape=(len(away_team_player), 2), dtype=np.float)
    
         
    # create array of x input data for every match
    x_intput = np.ndarray(shape=(data_height, data_width), dtype=np.float)        
            
    with open('../data/match_' + str(id) + '/per_sec_data_' + str(id) + '.json', 'r') as file:
        data_persec = json.load(file)
        
        
    # create first average positon data for assigning player position
    for t in data_persec:
        
        i = int(t['minute'])*60 + int(t['second'])
        x_pos = t['xpos']
        y_pos = t['ypos']
        team_id_t = int(t['teamId'])
        jersey_number_t = int(t['jerseyNumber'])
        has_ball_teamId_t = int(t['hasballTeamId'])
        
        if i==300:
            break
        
        first_5min = math.floor(239/60)
        
        
        
        if team_id_t == home_team_id:
            if player_positionId['homeTeam'][jersey_number_t] == 1:
                home_pos_count_5min[home_dict[str(jersey_number_t)]][0][first_5min] += x_pos
                home_pos_count_5min[home_dict[str(jersey_number_t)]][1][first_5min] += y_pos
                home_activity_count_5min[home_dict[str(jersey_number_t)]][first_5min] +=1 
            
            elif has_ball_teamId_t == away_team_id:
                home_pos_count_5min[home_dict[str(jersey_number_t)]][0][first_5min] += x_pos
                home_pos_count_5min[home_dict[str(jersey_number_t)]][1][first_5min] += y_pos
                home_activity_count_5min[home_dict[str(jersey_number_t)]][first_5min] +=1
        
        elif team_id_t == away_team_id:
            if player_positionId['awayTeam'][jersey_number_t] == 1:
                away_pos_count_5min[away_dict[str(jersey_number_t)]][0][first_5min] += x_pos
                away_pos_count_5min[away_dict[str(jersey_number_t)]][1][first_5min] += y_pos
                away_activity_count_5min[away_dict[str(jersey_number_t)]][first_5min] +=1 
            
            elif has_ball_teamId_t == home_team_id:
                away_pos_count_5min[away_dict[str(jersey_number_t)]][0][first_5min] += x_pos
                away_pos_count_5min[away_dict[str(jersey_number_t)]][1][first_5min] += y_pos
                away_activity_count_5min[away_dict[str(jersey_number_t)]][first_5min] +=1

            
    for player_home in range(0, len(home_team_player)):
        count = 0
        for minutes in range(0,5):
            home_pos_mean[player_home][0] += home_pos_count_5min[player_home][0][minutes]
            home_pos_mean[player_home][1] += home_pos_count_5min[player_home][1][minutes]
            count += home_activity_count_5min[player_home][minutes]

        if count!=0:
            home_pos_mean[player_home][0] /= count
            home_pos_mean[player_home][1] /= count
        else:
            home_pos_mean[player_home][0] = 0 
            home_pos_mean[player_home][1] = 0
            
    for player_away in range(0, len(away_team_player)):
        count = 0
        for minutes in range(0,5):
            away_pos_mean[player_away][0] += away_pos_count_5min[player_away][0][minutes]
            away_pos_mean[player_away][1] += away_pos_count_5min[player_away][1][minutes]
            count += away_activity_count_5min[player_away][minutes]

        if count!=0:
            away_pos_mean[player_away][0] /= count
            away_pos_mean[player_away][1] /= count
        else:
            away_pos_mean[player_away][0] = 0 
            away_pos_mean[player_away][1] = 0
            
                             
                                                      
    print(home_pos_mean)
    print(away_pos_mean)
    home_pos_mean = np.zeros(shape=(len(home_team_player), 2), dtype=np.float)
    away_pos_mean = np.zeros(shape=(len(away_team_player), 2), dtype=np.float)
    
    # temporary t        
    tmp_t = 0 
        
    for d in data_persec:
        
        
        i = int(d['minute'])*60 + int(d['second'])
        x_pos = d['xpos']
        y_pos = d['ypos']
        team_id_t = int(d['teamId'])
        jersey_number_t = int(d['jerseyNumber'])
        has_ball_teamId_t = int(d['hasballTeamId'])
        
        
        # shifing last index of 5min arry (period of 5min) with last min data
        # and calculate mena position of each player each min based on previus 5 min
        if (i+1)%60 == 0 and tmp_t != i+1 and i>299:
            
            tmp_t = i+1
            
            home_pos_mean = np.zeros(shape=(len(home_team_player), 2), dtype=np.float)
            away_pos_mean = np.zeros(shape=(len(away_team_player), 2), dtype=np.float)
            
            
            for player_home in range(0, len(home_team_player)):
                count = 0
                for minutes in range(0,5):
                    home_pos_mean[player_home][0] += home_pos_count_5min[player_home][0][minutes]
                    home_pos_mean[player_home][1] += home_pos_count_5min[player_home][1][minutes]
                    count += home_activity_count_5min[player_home][minutes]

                if count!=0:
                    home_pos_mean[player_home][0] /= count
                    home_pos_mean[player_home][1] /= count
                else:
                    home_pos_mean[player_home][0] = 0 
                    home_pos_mean[player_home][1] = 0
            
            for player_away in range(0, len(away_team_player)):
                count = 0
                for minutes in range(0,5):
                    away_pos_mean[player_away][0] += away_pos_count_5min[player_away][0][minutes]
                    away_pos_mean[player_away][1] += away_pos_count_5min[player_away][1][minutes]
                    count += away_activity_count_5min[player_away][minutes]

                if count!=0:
                    away_pos_mean[player_away][0] /= count
                    away_pos_mean[player_away][1] /= count
                else:
                    away_pos_mean[player_away][0] = 0 
                    away_pos_mean[player_away][1] = 0
            
                    
            print('\n')
            print(i+1)
            print(home_pos_mean)                         
            print(away_pos_mean)
            print('\n')
                    
            
            
            for player_home in range(0, len(home_team_player)):
                home_pos_count_5min[player_home][0] = (shift(home_pos_count_5min[player_home][0], -1, cval=0))
                home_pos_count_5min[player_home][1] = (shift(home_pos_count_5min[player_home][1], -1, cval=0))
                home_activity_count_5min[player_home] = (shift(home_activity_count_5min[player_home], -1, cval=0))

                home_pos_count_5min[player_home][0][4] = home_pos_count[player_home][0]
                home_pos_count_5min[player_home][1][4] = home_pos_count[player_home][1]
                home_activity_count_5min[player_home][4] = home_activity_count[player_home]
                
            for player_away in range(0, len(away_team_player)):
                away_pos_count_5min[player_away][0] = (shift(away_pos_count_5min[player_away][0], -1, cval=0))
                away_pos_count_5min[player_away][1] = (shift(away_pos_count_5min[player_away][1], -1, cval=0))
                away_activity_count_5min[player_away] = (shift(away_activity_count_5min[player_away], -1, cval=0))

                away_pos_count_5min[player_away][0][4] = away_pos_count[player_away][0]
                away_pos_count_5min[player_away][1][4] = away_pos_count[player_away][1]
                away_activity_count_5min[player_away][4] = away_activity_count[player_away]
                
            

            # clear home team
            home_pos_count = np.zeros(shape=(len(home_team_player), 2), dtype=np.float)
            home_activity_count = np.zeros(shape=(len(home_team_player), 1), dtype=np.int)


            # clear away team
            away_pos_count = np.zeros(shape=(len(away_team_player), 2), dtype=np.float)
            away_activity_count = np.zeros(shape=(len(away_team_player), 1), dtype=np.int)
            
    
        # getting players' and  referee's x,y coordinate on the pitch
        
        if team_id_t == home_team_id:
            if player_positionId['homeTeam'][jersey_number_t] == 1:
                home_pos_count[home_dict[str(jersey_number_t)]][0] += x_pos
                home_pos_count[home_dict[str(jersey_number_t)]][1] += y_pos
                home_activity_count[home_dict[str(jersey_number_t)]] +=1  
            
            elif has_ball_teamId_t == away_team_id:
                home_pos_count[home_dict[str(jersey_number_t)]][0] += x_pos
                home_pos_count[home_dict[str(jersey_number_t)]][1] += y_pos
                home_activity_count[home_dict[str(jersey_number_t)]] +=1 

        elif team_id_t == away_team_id:
            if player_positionId['awayTeam'][jersey_number_t] == 1:
                away_pos_count[away_dict[str(jersey_number_t)]][0] += x_pos
                away_pos_count[away_dict[str(jersey_number_t)]][1] += y_pos
                away_activity_count[away_dict[str(jersey_number_t)]] +=1  
            
            elif has_ball_teamId_t == home_team_id:
                away_pos_count[away_dict[str(jersey_number_t)]][0] += x_pos
                away_pos_count[away_dict[str(jersey_number_t)]][1] += y_pos
                away_activity_count[away_dict[str(jersey_number_t)]] +=1 

        elif team_id_t == referee_id:
            pass
            # referee_pos_t['xpos']
            # referee_pos_t['ypos']
        
        
        if has_ball_teamId_t == 0:
            pass
        else: 
            pass
        
    

[[16.8264934  33.72919252]
 [ 0.          0.        ]
 [34.02382145 34.19083905]
 [34.89217054 22.26637903]
 [48.73119773 45.64969899]
 [48.05663084 10.62662984]
 [ 0.          0.        ]
 [47.47121884 25.9716615 ]
 [ 0.          0.        ]
 [42.01236358 29.7873194 ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [61.07408143 27.87044228]
 [ 0.          0.        ]
 [36.88420387 46.17592741]
 [ 0.          0.        ]
 [37.1684112  12.1421558 ]
 [54.08478702 27.74704294]]
[[ 0.          0.        ]
 [71.9778761  40.72925286]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [49.84658122 48.40210497]
 [40.30391755 23.99085159]
 [56.04199593 16.54005743]
 [61.69152073 39.7460574 ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [68.06366358 39.84534915]
 [70.79587624 28.39368121]
 [94.19874507 32.81441254]
 [70.80076204 20.68199779]




1260
[[11.51756623 33.87351666]
 [ 0.          0.        ]
 [39.77060501 34.2839221 ]
 [39.6130641  23.66825663]
 [59.07918393 49.17301268]
 [55.57321317 15.15110771]
 [ 0.          0.        ]
 [50.78949341 36.24552351]
 [ 0.          0.        ]
 [54.58439047 27.77325593]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [69.45499767 27.61953304]
 [ 0.          0.        ]
 [47.90421569 47.18745073]
 [ 0.          0.        ]
 [44.06389011 13.4921449 ]
 [64.48781718 34.51666291]]
[[ 0.          0.        ]
 [61.69176714 37.63179034]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [44.2248652  33.54138583]
 [37.63015121 30.4694458 ]
 [39.54009313 12.33859636]
 [42.96715121 40.35876426]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [64.47820506 49.07951146]
 [62.15864556 20.50957743]
 [85.61880037 32.20618593]
 [43.28744233  9.791



2340
[[16.68049655 35.09112203]
 [ 0.          0.        ]
 [28.93303696 36.73036436]
 [36.71779868 18.56788911]
 [36.58978614 50.04954059]
 [26.05639175 15.24570729]
 [ 0.          0.        ]
 [37.72794983 38.85373003]
 [ 0.          0.        ]
 [35.04931089 29.95084752]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [53.75106931 34.25660198]
 [ 0.          0.        ]
 [26.81661914 48.48525017]
 [ 0.          0.        ]
 [27.01331353 25.19823036]
 [47.10591947 41.97946271]]
[[ 0.          0.        ]
 [71.61623648 42.57301509]
 [ 0.          0.        ]
 [57.19636508 40.94115873]
 [56.93188428 33.22170189]
 [46.85179371 21.56001509]
 [50.6586355  23.81162767]
 [52.30865409 52.78173711]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [63.7594566  51.47144403]
 [63.20622893 26.05690818]
 [91.00776803 34.68181992]
 [61.03153082 20.871



3180
[[ 7.25901035 34.03021837]
 [ 0.          0.        ]
 [24.36692237 39.41234607]
 [27.16210557 24.72963071]
 [35.60629063 49.61932978]
 [34.62077206 13.49636159]
 [ 0.          0.        ]
 [35.88332365 34.43080278]
 [ 0.          0.        ]
 [33.06827892 34.82196363]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [46.87834469 33.51262911]
 [ 0.          0.        ]
 [25.36492194 47.3028523 ]
 [ 0.          0.        ]
 [23.03075956 17.60851892]
 [36.62411423 30.53209676]]
[[ 0.          0.        ]
 [60.45231562 43.33243676]
 [ 0.          0.        ]
 [54.04517505 31.85096425]
 [42.16499682 31.20465095]
 [30.66370559 29.17362037]
 [37.47042467 15.49244504]
 [35.43075817 45.94469673]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [53.42179764 53.74273023]
 [56.32324514 34.05003741]
 [81.68025706 34.19163382]
 [54.0806347  16.156



4320
[[13.48195679 32.95365668]
 [ 0.          0.        ]
 [29.82338    33.7521181 ]
 [25.1206219  27.84942381]
 [36.66614149  6.07197428]
 [32.63513429 27.51155524]
 [ 0.          0.        ]
 [32.14481905 27.40158667]
 [ 0.          0.        ]
 [37.23361475 24.14258914]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [37.22336571 34.71946952]
 [ 0.          0.        ]
 [26.02798667 34.11589619]
 [ 0.          0.        ]
 [26.32734138 14.15468357]
 [23.49447619 22.83979619]]
[[ 0.          0.        ]
 [67.34828517 29.2935562 ]
 [ 0.          0.        ]
 [58.25672227 34.76531286]
 [58.62837689 37.68787116]
 [41.32354924 33.83359105]
 [59.78454337 21.77829273]
 [47.52558374 17.92239486]
 [54.00124435 50.6997839 ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [67.55352352 50.25047372]
 [69.77217534 23.88790031]
 [89.82815539 30.86104779]
 [68.86045198 20.172



5340
[[ 7.4530936  35.8311966 ]
 [ 0.          0.        ]
 [26.39043178 37.56987287]
 [27.67020233 27.58416357]
 [ 0.          0.        ]
 [38.16586512 50.60854419]
 [50.00347524 35.3653953 ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [32.18262481 31.03196977]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [41.42657984 27.39026279]
 [34.66723256 33.72585659]
 [43.38988931 25.35818868]
 [ 0.          0.        ]
 [27.04147364 45.83552868]
 [ 0.          0.        ]
 [25.91513178 18.09319845]
 [37.06339535 16.93379922]]
[[ 0.          0.        ]
 [40.8100631  33.36414405]
 [ 0.          0.        ]
 [50.94993214 37.7392131 ]
 [48.85202127 36.10622153]
 [38.73946905 35.83771548]
 [40.57279643 22.21580476]
 [ 0.          0.        ]
 [41.9614881  55.90082262]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [55.24064762 53.6707881 ]
 [55.76919048 29.95194167]
 [81.66760273 33.22811435]
 [56.64895238 17.680